In [59]:
import scanpy as sc
import re

In [222]:
adata = sc.read("../../data/adamson.h5ad")

# print(adata.obs["UMI count"].head())
# print(adata.var)

In [223]:
adata.obs["perturbation"].value_counts()

perturbation
3x_neg_ctrl_pMJ144-1     1715
ATF6_PERK_IRE1_pMJ158    1693
ATF6_only_pMJ145         1691
IRE1_only_pMJ148         1685
3x_neg_ctrl_pMJ144-2     1650
PERK_IRE1_pMJ154         1629
ATF6_PERK_pMJ150         1603
PERK_only_pMJ146         1560
ATF6_IRE1_pMJ152         1449
*                          13
PSMD12_pDS009               6
PSMA1_pDS007                4
IER3IP1_pDS003              3
C7orf26_pDS004              2
XBP1_pBA578                 2
Gal4-4(mod)_pBA582          1
ATF4_pBA576                 1
SNAI1_pDS266                1
XBP1_pBA579                 1
YIPF5_pDS001                1
Name: count, dtype: int64

In [62]:
adata

AnnData object with n_obs × n_vars = 15006 × 32738
    obs: 'perturbation', 'read count', 'UMI count', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts'
    var: 'ensembl_id', 'ncounts', 'ncells'

In [86]:
# Feature selection for highly variable genes, feel free to modify parameters

sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=2000, layer="counts")

In [87]:
adata = adata[:,adata.var["highly_variable"]]

In [65]:
adata

View of AnnData object with n_obs × n_vars = 15006 × 2000
    obs: 'perturbation', 'read count', 'UMI count', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'n_genes'
    var: 'ensembl_id', 'ncounts', 'ncells', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'log1p', 'hvg'
    layers: 'counts'

In [88]:
adata = adata[~adata.obs["perturbation"].isna() & (adata.obs["perturbation"] != "*")].copy()
# adata = adata[~adata.obs["perturbation"].str.contains(r"ctrl|control", flags=re.IGNORECASE, na=False)].copy()
adata.obs["perturbation"].unique()

['3x_neg_ctrl_pMJ144-1', '3x_neg_ctrl_pMJ144-2', 'ATF6_PERK_IRE1_pMJ158', 'ATF6_PERK_pMJ150', 'ATF6_only_pMJ145', ..., 'ATF4_pBA576', 'SNAI1_pDS266', 'XBP1_pBA579', 'Gal4-4(mod)_pBA582', 'YIPF5_pDS001']
Length: 19
Categories (19, object): ['3x_neg_ctrl_pMJ144-1', '3x_neg_ctrl_pMJ144-2', 'ATF4_pBA576', 'ATF6_IRE1_pMJ152', ..., 'SNAI1_pDS266', 'XBP1_pBA578', 'XBP1_pBA579', 'YIPF5_pDS001']

In [224]:
%run ../utils.py
split = {"train": 0.7, "val": 0.15, "test": 0.15}
assign_splits_proportional(adata, split)

chose PERK_only_pMJ146, so now test_perts_count=1560
chose ATF6_PERK_pMJ150, so now test_perts_count=3163


In [234]:
print(adata.obs["split"].value_counts())
print("perts in train", list(adata.obs.loc[adata.obs["split"] == "train", "perturbation"].unique()))
print("perts in test", list(adata.obs.loc[adata.obs["split"] == "test", "perturbation"].unique()))
print("perts in val", list(adata.obs.loc[adata.obs["split"] == "val", "perturbation"].unique()))

split
train    9502
test     3163
val      2341
Name: count, dtype: int64
perts in train ['3x_neg_ctrl_pMJ144-1', '3x_neg_ctrl_pMJ144-2', 'ATF6_PERK_IRE1_pMJ158', 'ATF6_only_pMJ145', 'PERK_IRE1_pMJ154', 'ATF6_IRE1_pMJ152', 'IRE1_only_pMJ148', '*', nan, 'IER3IP1_pDS003', 'PSMA1_pDS007', 'PSMD12_pDS009', 'C7orf26_pDS004', 'ATF4_pBA576', 'SNAI1_pDS266', 'XBP1_pBA579', 'Gal4-4(mod)_pBA582', 'YIPF5_pDS001']
perts in test ['ATF6_PERK_pMJ150', 'PERK_only_pMJ146']
perts in val ['ATF6_IRE1_pMJ152', 'ATF6_only_pMJ145', 'ATF6_PERK_IRE1_pMJ158', 'IRE1_only_pMJ148', 'PERK_IRE1_pMJ154', nan, 'XBP1_pBA578', '*', 'PSMA1_pDS007', 'PSMD12_pDS009']


In [69]:
if "log1p" in adata.uns:
    del adata.uns["log1p"]
adata

AnnData object with n_obs × n_vars = 11332 × 2000
    obs: 'perturbation', 'read count', 'UMI count', 'tissue_type', 'cell_line', 'cancer', 'disease', 'perturbation_type', 'celltype', 'organism', 'ncounts', 'ngenes', 'percent_mito', 'percent_ribo', 'nperts', 'n_genes', 'split'
    var: 'ensembl_id', 'ncounts', 'ncells', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg'
    layers: 'counts'

In [99]:
perts = list(adata.obs["perturbation"].unique())
pert_map = {}
for pert in perts:
    pert_split = [gene for gene in pert.split("_")[:-1] if gene != 'only']
    pert_map[pert] = pert_split
print(pert_map)

{'3x_neg_ctrl_pMJ144-1': ['3x', 'neg', 'ctrl'], '3x_neg_ctrl_pMJ144-2': ['3x', 'neg', 'ctrl'], 'ATF6_PERK_IRE1_pMJ158': ['ATF6', 'PERK', 'IRE1'], 'ATF6_PERK_pMJ150': ['ATF6', 'PERK'], 'ATF6_only_pMJ145': ['ATF6'], 'PERK_IRE1_pMJ154': ['PERK', 'IRE1'], 'ATF6_IRE1_pMJ152': ['ATF6', 'IRE1'], 'IRE1_only_pMJ148': ['IRE1'], 'PERK_only_pMJ146': ['PERK'], 'XBP1_pBA578': ['XBP1'], 'IER3IP1_pDS003': ['IER3IP1'], 'PSMA1_pDS007': ['PSMA1'], 'PSMD12_pDS009': ['PSMD12'], 'C7orf26_pDS004': ['C7orf26'], 'ATF4_pBA576': ['ATF4'], 'SNAI1_pDS266': ['SNAI1'], 'XBP1_pBA579': ['XBP1'], 'Gal4-4(mod)_pBA582': ['Gal4-4(mod)'], 'YIPF5_pDS001': ['YIPF5']}


In [74]:
# adata = sc.read("../../data/preprocessed/adamson_preprocessed.h5ad")
adata.obs["perturbation"].value_counts()

perturbation
ATF6_PERK_IRE1_pMJ158    1693
ATF6_only_pMJ145         1691
IRE1_only_pMJ148         1685
PERK_IRE1_pMJ154         1629
ATF6_PERK_pMJ150         1603
PERK_only_pMJ146         1560
ATF6_IRE1_pMJ152         1449
PSMD12_pDS009               6
PSMA1_pDS007                4
IER3IP1_pDS003              3
XBP1_pBA578                 2
C7orf26_pDS004              2
XBP1_pBA579                 1
ATF4_pBA576                 1
SNAI1_pDS266                1
Gal4-4(mod)_pBA582          1
YIPF5_pDS001                1
Name: count, dtype: int64